In [1]:
# metadata needed - assume user provides as much as possible as pre-processing step, library fills in the rest
    # instrument
    # tempo
    # key
    # percussive / harmonic / vocal --> sound_class

# so our library ultimately takes a stem and its respective metadata folder as input

# note: i think library should choose percussive stem as base 

# now we go through 4 cases: if instrument is missing, if tempo is missing, if key is missing, if percussive / harmonic / vocal is missing

In [2]:
import os
import json
import uuid
import soundfile as sf
import librosa
import glob
import random
from IPython.display import Audio
import numpy as np

In [3]:
# brid-preprocessing
# need to assign null in all cases, don't want any empty variables. assign null in "other" folders

def get_instrument_name(file_path):

    instrument_folders = {
            "-PD" : "pandeiro",
            "-TB": "tamborim",
            "-RR": "reco-reco",
            "-CX": "caixa",
            "-RP": "repique",
            "-CU": "cuica",
            "-AG": "agogo",
            "-SK": "shaker",
            "-TT": "tanta",
            "-SU": "surdo"}
    for key in instrument_folders:
        if key in file_path:
            return instrument_folders[key]
            
    return None

def get_tempo(file_path):
    
    folders = ["samba","marcha","partido alto","samba-enredo","other","capoeira"]
    
    style_folders = {
        "SA.wav" : "samba",
        "PA.wav": "partido alto",
        "CA.wav": "capoeira",
        "SE.wav": "samba-enredo",
        "MA.wav": "marcha",
        "OT.wav": "other"
    }

    style_to_tempo = {
        "samba" : 80.0,
        "partido alto" : 100.0,
        "samba-enredo" : 130.0,
        "marcha" : 120.0,
        "capoeira" : 65.0
    }
    
    style = None
    
    for suffix in style_folders:
        if suffix in file_path:
            style = style_folders[suffix]
            
    if style is None:
        return None

    if style in style_to_tempo:
        return style_to_tempo[style]

    return None

In [4]:
test_path = os.path.join("/Users/lgpietrewicz/Documents/stem_mixer/stems", "[0111] S1-PD3-04-MA.wav")

In [5]:
tempo = get_tempo(test_path)
instrument_name = get_instrument_name(test_path)
key = None
sound_class = "percussive"

In [6]:
# musdb pre-processing 

def get_type_name(file_path):
    
    type_folders = ["vocals", "drums", "bass", "other"]
    
    for name in type_folders:
        if name in file_path:
            return name
            
    return None

def get_sound_class(instrument_name):
    if instrument_name == "vocals":
        sound_class = "vocals"
    elif instrument_name == "drums":
        sound_class = "percussive"
    elif instrument_name == "bass" or instrument_name == "other":
        sound_class = "harmonic"
    else:
        sound_class = None

    return sound_class

In [7]:
tempo = None
instrument_name = get_type_name(test_path)
key = None 
sound_class = get_sound_class(test_path)

To rename a file in Python, you can use the os. rename() function from the os module. The os. rename() function takes two arguments: the current name of the file or directory (source), and the new name (destination).

In [8]:
test_path = os.path.join("/Users/lgpietrewicz/Documents/stem_mixer/stems", "Cristina Vane - So Easy - vocals.wav")

In [9]:
def metadata_extraction(path_to_stems, tempo = None, instrument_name = None, key = None, sound_class = None):
    for root, dirs, files in os.walk(path_to_stems):
        for file in files:
            file_path = os.path.join(root, file)

            if ".json" in file_path:
                continue
                
            if ".wav" or ".mp3" in file_path:
                json_name = file_path[0:len(file_path)-4]
            else:
                json_name = file_path
        
            metadata = {}
            metadata["tempo"] = tempo
            metadata["instrument_name"] = instrument_name
            metadata["key"] = key
            metadata["sound_class"] = sound_class
        
            json_file_path = os.path.join(path_to_stems, f"{json_name}.json")
            
            with open(json_file_path, "w") as json_file:
                    json.dump(metadata, json_file, indent=4)

In [29]:
home = os.getcwd()
test_path = os.path.join(home, "stems")

In [109]:
metadata_extraction(test_path, tempo = None, sound_class = None)

In [110]:
# overview of process: fill in "None" values. might need to do research --> one method might be best for something like drums,
# other methods might work best for something like vocals. need to make sure all values are locked in before i start worrying about 
# generating mixtures 

In [111]:
# generating mixtures, ignoring all conditions / parameters
# send these to output folder

# want mixture outputs to be folders w n stems and mixture .wav or .mp3 files

def generate_mixture(data_home, sr, n_stems = 4, base_stem = None, percussive_to_harmonic = None):

    print(data_home)
    # make folder to store all mixtures
    mixture_folder = os.path.join(data_home, "mixtures")
    os.makedirs(mixture_folder, exist_ok = True)

    path_to_stems = os.path.join(data_home, "stems")

    all_files = [os.path.join(path_to_stems, f) for f in os.listdir(path_to_stems) if os.path.isfile(os.path.join(path_to_stems, f))]

    audio_files = []
    for file in all_files:
        if ".json" not in file:
            audio_files.append(file)
            
    if len(audio_files) < n_stems:
        raise ValueError(f"Not enough files in the directory to choose {num_files} random files.")
    
    # select random files
    random_files = random.sample(audio_files, n_stems)

    print(random_files)

    stem_audios = []
    stem_audio_lengths = []
    
    for stem in random_files:

        stem_audio, sr = librosa.load(stem, sr = sr)
        stem_audio_len = len(stem_audio)
            
        stem_audios.append(stem_audio)
        stem_audio_lengths.append(stem_audio_len)
        
        # display(Audio(stem_audio, rate=sr))

    min_length = min(stem_audio_lengths)
    min_pos = stem_audio_lengths.index(min_length)
    mixture_audio = stem_audios[min_pos]

    truncated_stems = []
    for audio in stem_audios:
        audio = audio[:min_length]
        truncated_stems.append(audio)

    # give each mixture a random id
    mixture_id = str(uuid.uuid4())
    individual_output_folder = os.path.join(mixture_folder, mixture_id)
    os.makedirs(individual_output_folder, exist_ok = True)
    
    for k in range(0, len(truncated_stems)):
        sf.write(f"{individual_output_folder}/stem{k+1}.wav", truncated_stems[k], sr)
        if(k != min_pos): # already accounted for
            mixture_audio = mixture_audio + truncated_stems[k]

    sf.write(f"{individual_output_folder}/mixture.wav", mixture_audio, sr)


In [284]:
home = os.getcwd()
sr = 41500

generate_mixture(home, sr)

/Users/lgpietrewicz/Documents/stem_mixer
['/Users/lgpietrewicz/Documents/stem_mixer/stems/[0189] S2-PD2-05-CA.wav', '/Users/lgpietrewicz/Documents/stem_mixer/stems/Bobby Nobody - Stitch Up - vocals.wav', '/Users/lgpietrewicz/Documents/stem_mixer/stems/[0111] S1-PD3-04-MA.wav', '/Users/lgpietrewicz/Documents/stem_mixer/stems/The Long Wait - Back Home To Blue - bass.wav']


In [118]:
# now we want code to fill in tempo (i suppose i am doing this one track at a time?)
def set_tempo(data_home, sr):
    path_to_stems = os.path.join(data_home, "stems")
    json_files = [file for file in os.listdir(path_to_stems) if file.endswith(".json")]

    # file_names = []
    for json_file in json_files:
        file_path = os.path.join(path_to_stems, json_file)
        file_name = os.path.basename(file_path)
        file_name = os.path.splitext(file_name)[0]

        print(file_name)
        # file_names.append(file_name)
        
        with open(file_path, "r") as file:
            data = json.load(file)
            print(data)
        
        if "tempo" in data:
            if data["tempo"] is None:
                print(f"Tempo is None in file: {json_file}")

                try:
                    audio_path = os.path.join(path_to_stems, get_wav_from_json(path_to_stems, file_name))
                    audio_file, sr = librosa.load(audio_path, sr = sr)
                    tempo, _ = librosa.beat.beat_track(y=audio_file, sr=sr)
                    tempo = float(tempo[0])
    
                    print(type(tempo))
                    data["tempo"] = tempo
    
                    with open(file_path, "w") as file:
                        json.dump(data, file, indent=4)
                        
                except FileNotFoundError as e:
                    print(f"File not found: {file_path}")
                except Exception as e:
                    print(f"Unexpected error: {e}")
                    
                    
            else:
                print(f"Tempo in {json_file}: {data['tempo']}")
        else:
            print(f"Tempo not found in file: {json_file}")


In [119]:
def get_wav_from_json(path_to_stems, file_name): # useful for getting tempo, instrument name, etc
    for file in os.listdir(path_to_stems):
        if file_name in file and not file.endswith(".json"):
            audio_path = file
    return audio_path

In [120]:
sr = 44100
set_tempo(home, sr)

[0189] S2-PD2-05-CA
{'tempo': 129.19921875, 'instrument_name': None, 'key': None, 'sound_class': None}
Tempo in [0189] S2-PD2-05-CA.json: 129.19921875
[0111] S1-PD3-04-MA
{'tempo': 120.18531976744185, 'instrument_name': None, 'key': None, 'sound_class': None}
Tempo in [0111] S1-PD3-04-MA.json: 120.18531976744185
[0273] S2-SU2-04-VPA
{'tempo': 132.51201923076923, 'instrument_name': None, 'key': None, 'sound_class': None}
Tempo in [0273] S2-SU2-04-VPA.json: 132.51201923076923
[0314] S3-RR1-07-OT
{'tempo': 105.46875, 'instrument_name': None, 'key': None, 'sound_class': None}
Tempo in [0314] S3-RR1-07-OT.json: 105.46875
.Music Delta - Rock - bass.wav.ic
{'tempo': None, 'instrument_name': None, 'key': None, 'sound_class': None}
Tempo is None in file: .Music Delta - Rock - bass.wav.ic.json


/var/folders/6h/xwflp2n17lb7zh8dz7v3zh2m0000gn/T/ipykernel_23595/3113494595.py:25: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_file, sr = librosa.load(audio_path, sr = sr)
/Users/lgpietrewicz/anaconda3/envs/reu/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Unexpected error: 
.DS_S
{'tempo': None, 'instrument_name': None, 'key': None, 'sound_class': None}
Tempo is None in file: .DS_S.json
Unexpected error: 
The Long Wait - Back Home To Blue - bass
{'tempo': 80, 'instrument_name': None, 'key': None, 'sound_class': 'hi'}
Tempo in The Long Wait - Back Home To Blue - bass.json: 80
.The Long Wait - Back Home To Blue - bass.wav.ic
{'tempo': None, 'instrument_name': None, 'key': None, 'sound_class': None}
Tempo is None in file: .The Long Wait - Back Home To Blue - bass.wav.ic.json
Unexpected error: 
.The Wrong'Uns - Rothko - other.wav.ic
{'tempo': None, 'instrument_name': None, 'key': None, 'sound_class': None}
Tempo is None in file: .The Wrong'Uns - Rothko - other.wav.ic.json
Unexpected error: 
.[0257] S2-SK2-01-SA.wav.ic
{'tempo': None, 'instrument_name': None, 'key': None, 'sound_class': None}
Tempo is None in file: .[0257] S2-SK2-01-SA.wav.ic.json
Unexpected error: 
[0307] S3-TB3-02-VSE
{'tempo': None, 'instrument_name': None, 'key': None, '

/var/folders/6h/xwflp2n17lb7zh8dz7v3zh2m0000gn/T/ipykernel_23595/3113494595.py:25: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_file, sr = librosa.load(audio_path, sr = sr)
/Users/lgpietrewicz/anaconda3/envs/reu/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/var/folders/6h/xwflp2n17lb7zh8dz7v3zh2m0000gn/T/ipykernel_23595/3113494595.py:25: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_file, sr = librosa.load(audio_path, sr = sr)
/Users/lgpietrewicz/anaconda3/envs/reu/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/var/folders/6h/

<class 'float'>
PR - Happy Daze - bass
{'tempo': None, 'instrument_name': None, 'key': None, 'sound_class': None}
Tempo is None in file: PR - Happy Daze - bass.json
<class 'float'>
[0316] S3-CX1-01-SA
{'tempo': None, 'instrument_name': None, 'key': None, 'sound_class': None}
Tempo is None in file: [0316] S3-CX1-01-SA.json
<class 'float'>
Al James - Schoolboy Facination - drums
{'tempo': None, 'instrument_name': None, 'key': None, 'sound_class': None}
Tempo is None in file: Al James - Schoolboy Facination - drums.json
<class 'float'>
Titanium - Haunted Age - other
{'tempo': 80, 'instrument_name': None, 'key': None, 'sound_class': 'hi'}
Tempo in Titanium - Haunted Age - other.json: 80
.Titanium - Haunted Age - other.wav.ic
{'tempo': None, 'instrument_name': None, 'key': None, 'sound_class': None}
Tempo is None in file: .Titanium - Haunted Age - other.wav.ic.json
Unexpected error: 
[0338] S3-AG1-03-SE
{'tempo': None, 'instrument_name': None, 'key': None, 'sound_class': None}
Tempo is None

/var/folders/6h/xwflp2n17lb7zh8dz7v3zh2m0000gn/T/ipykernel_23595/3113494595.py:25: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_file, sr = librosa.load(audio_path, sr = sr)
/Users/lgpietrewicz/anaconda3/envs/reu/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


<class 'float'>
The Wrong'Uns - Rothko - other
{'tempo': 80, 'instrument_name': None, 'key': None, 'sound_class': 'hi'}
Tempo in The Wrong'Uns - Rothko - other.json: 80
Bobby Nobody - Stitch Up - vocals
{'tempo': None, 'instrument_name': None, 'key': None, 'sound_class': None}
Tempo is None in file: Bobby Nobody - Stitch Up - vocals.json
<class 'float'>
[0240] S2-RP1-04-VPA
{'tempo': None, 'instrument_name': None, 'key': None, 'sound_class': None}
Tempo is None in file: [0240] S2-RP1-04-VPA.json
<class 'float'>
Motor Tapes - Shore - drums
{'tempo': None, 'instrument_name': None, 'key': None, 'sound_class': None}
Tempo is None in file: Motor Tapes - Shore - drums.json
<class 'float'>
